In [77]:
import numpy as numpy
import streamlit as streamlit 
import pandas as pandas
import random  
import matplotlib.pyplot as plt
import seaborn as seaborn
from faker import Faker  
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

fake = Faker()

num_transactions = 100

data = []
for _ in range(num_transactions):
    transaction = {
        "transaction_id": fake.uuid4(),
        "user_id": fake.random_int(min=1000, max=2000),
        "amount": round(random.uniform(10, 5000), 2),
        "transaction_type": random.choice(["Online Transaction", "ATM Transaction", "Bank Transaction", "Bill Payment"]),
        "timestamp": fake.date_time(),  
        "location": fake.city(),
        "account_balance": round(random.uniform(100, 20000), 2),
        "fraudulent": random.choice([0, 1])  
    }
    data.append(transaction)

df = pandas.DataFrame(data)

df["timestamp"] = pandas.to_datetime(df["timestamp"])

df["year"] = df["timestamp"].dt.year
df["month"] = df["timestamp"].dt.month
df["day"] = df["timestamp"].dt.day
df["hour"] = df["timestamp"].dt.hour  
df["minute"] = df["timestamp"].dt.minute
df["second"] = df["timestamp"].dt.second
df["day_of_week"] = df["timestamp"].dt.weekday 
df["is_weekend"] = df["day_of_week"].apply(lambda x: 1 if x >= 5 else 0) 

df.drop(columns=["timestamp"], inplace=True)

df.to_csv("fake_transactions.csv", index=False)

df = pandas.read_csv("fake_transactions.csv")

df.columns = df.columns.str.strip().str.lower()

streamlit.write("Preview of first 5 entries:", df.head())

feature_columns = ["amount", "transaction_type", "account_balance", "year", "month", "day", "hour", "minute", "second", "day_of_week", "is_weekend"]

if all(col in df.columns for col in feature_columns):
    X = df[feature_columns].copy()
    X = pd.get_dummies(X, columns=["transaction_type"], drop_first=True)

    y = df["fraudulent"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)

    st.write(f"Model training completed! Accuracy: {accuracy:.2f}")
    st.text("Classification Report:")
    st.text(classification_rep)

else:
    missing_columns = [col for col in feature_columns if col not in df.columns]
    st.error(f"There are columns {missing_columns} missing! The dataset needs updated.")


2025-03-11 08:32:49.849 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 08:32:49.850 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 08:32:49.851 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 08:32:49.851 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 08:32:49.866 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 08:32:49.867 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 08:32:49.963 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 08:32:49.963 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar